In [2]:
# importing libraries
from nltk.util import pr
from collections import Counter
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from natsort import natsorted
import string
import os
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from yaml.tokens import Token
import nltk

## Part 1 

In [31]:

path = r"C:/Users/Yousef Khaled/Documents/GitHub/Information_retrival/New folder"
os.chdir(path)

In [32]:
## insert paths in dataset 
stemmer= PorterStemmer()
File_Map=[]
pos_index = {}
file_map={}
for file in os.listdir():

    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
        File_Map.append(file_path)
       

In [33]:
File_Map

['C:/Users/Yousef Khaled/Documents/GitHub/Information_retrival/New folder/1.txt',
 'C:/Users/Yousef Khaled/Documents/GitHub/Information_retrival/New folder/2.txt',
 'C:/Users/Yousef Khaled/Documents/GitHub/Information_retrival/New folder/3.txt',
 'C:/Users/Yousef Khaled/Documents/GitHub/Information_retrival/New folder/4.txt',
 'C:/Users/Yousef Khaled/Documents/GitHub/Information_retrival/New folder/5.txt',
 'C:/Users/Yousef Khaled/Documents/GitHub/Information_retrival/New folder/6.txt',
 'C:/Users/Yousef Khaled/Documents/GitHub/Information_retrival/New folder/7.txt',
 'C:/Users/Yousef Khaled/Documents/GitHub/Information_retrival/New folder/8.txt']

In [34]:
# Number of documents used
N=len(File_Map)


In [35]:
N

8

## Preprocessing

In [36]:
def convert_lower_case(data):
    return np.char.lower(data)

In [37]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [38]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [39]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [40]:
# def stemming(data):
#     tokens = word_tokenize(str(data))
#     new_text = ""
#     for w in tokens:
#         new_text = new_text + " " + stemmer.stem(w)
#     return new_text

In [41]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) 
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    return data

## Extraction

In [42]:
# read file from pathes in dataset and extract terms after prepocessing
processed_text = []

for i in File_Map:
    file = open(i, 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))


In [43]:
processed_text

[['president', 'talks', 'uconn'],
 ['apple', 'stock', 'hot'],
 ['born', 'ideals', 'dreams'],
 ['born', 'greatness'],
 ['born', 'wings'],
 ['meant', 'crawling', 'dont'],
 ['wings'],
 ['learn', 'use', 'fly']]

## Part 2

In [44]:
def positional_index():
    fileno = 0
    i=0
    for file_name in File_Map:
            
            text = processed_text[i]
            #print(text)

            for pos, term in enumerate(text):
                #term = stemmer.stem(term)

                if term in pos_index:
                    pos_index[term][0] = pos_index[term][0] + 1

                    if fileno in pos_index[term][1]:
                        pos_index[term][1][fileno].append(pos)

                    else:
                        pos_index[term][1][fileno] = [pos]

                else:
                    pos_index[term] = []
                    pos_index[term].append(1)
                    pos_index[term].append({})
                    pos_index[term][1][fileno] = [pos]
                
            i=i+1		
            file_map[fileno] = File_Map[fileno]
            fileno += 1


In [17]:
positional_index()

In [45]:
input_user = input("\n please write phrase query on positional index \n ")  

try:
    sample_pos_idx = pos_index[input_user]
    print("\n Positional Index")
    print(sample_pos_idx)

    file_list = sample_pos_idx[1]

    print("\n Filename, \t [Positions] \n ")
    for fileno, positions in file_list.items():
        print(file_map[fileno] , positions)
        print("\n")
except KeyError:
    print("this word does't exist ")
    


 please write phrase query on positional index 
 born
this word does't exist 


### Part 3

In [26]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

for i in DF:
    DF[i] = len(DF[i])

In [30]:
DF

{'president': 1,
 'talks': 1,
 'uconn': 1,
 'apple': 1,
 'stock': 1,
 'hot': 2,
 'job': 1,
 'market': 1}

In [27]:
def document_f(word):
    doc_number = 0
    try:
        c = DF[word]
    except:
        pass
    return c

### Calculate  Term Frequency and IDF 

In [29]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens )
    words_count = len(tokens )
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        
        df = document_f(token)
        print(df)
        idf = np.log(N/(df))
        print(token+' in document '+str(doc+1)+' : term frequence is  :  '+str(tf)+'\n IDF is : ' +str(idf))
        tf_idf[doc, token] = tf*idf

    doc += 1

1
president in document 1 : term frequence is  :  0.3333333333333333
 IDF is : 1.0986122886681098
1
talks in document 1 : term frequence is  :  0.3333333333333333
 IDF is : 1.0986122886681098
1
uconn in document 1 : term frequence is  :  0.3333333333333333
 IDF is : 1.0986122886681098
1
apple in document 2 : term frequence is  :  0.3333333333333333
 IDF is : 1.0986122886681098
2
hot in document 2 : term frequence is  :  0.3333333333333333
 IDF is : 0.4054651081081644
1
stock in document 2 : term frequence is  :  0.3333333333333333
 IDF is : 1.0986122886681098
2
hot in document 3 : term frequence is  :  0.5
 IDF is : 0.4054651081081644
1
job in document 3 : term frequence is  :  0.25
 IDF is : 1.0986122886681098
1
market in document 3 : term frequence is  :  0.25
 IDF is : 1.0986122886681098


### Display tf.IDF Matrix

In [22]:
x=list(tf_idf.keys())

In [23]:
doc_list=[]
term_list=[]
for i in range(len(x)):
    doc_list.append(x[i][0])
    term_list.append(x[i][1])
term_list=np.unique(term_list) 
doc_list=np.unique(doc_list)

In [24]:
import pandas as pd
columns=[]
columns.append('term')
for i in doc_list:
    columns.append('doc'+str(i+1))
    
TF_ID_matrix=pd.DataFrame(columns=columns,index=None)
TF_ID_matrix['term']=term_list
for i in range(len(TF_ID_matrix)):
    for j in doc_list:
        first_key=j
        column='doc'+str(j+1)
        second_key=TF_ID_matrix['term'].iloc[i]
        if(first_key, second_key)in tf_idf.keys():
            TF_ID_matrix[column].iloc[i]=tf_idf[first_key,second_key]
        else:
            continue
TF_ID_matrix.fillna(0,inplace=True)            

TF_ID_matrix.to_csv('TF_ID_matrix.csv',index=None)

In [25]:
TF_ID_matrix.head(20) 

,term,doc1,doc2,doc3
0,apple,0.00000,0.159040,0.000000
1,hot,0.00000,0.058697,0.088046
2,job,0.00000,0.000000,0.119280
3,market,0.00000,0.000000,0.119280
4,president,0.15904,0.000000,0.000000
5,stock,0.00000,0.159040,0.000000
6,talks,0.15904,0.000000,0.000000
7,uconn,0.15904,0.000000,0.000000
